In [5]:
import glob
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle

from keras.preprocessing import sequence
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [6]:
look_back = 200
look_forward = 10
units = 200

In [7]:
with open('../Datapreprocessing/Data/No Smoothing/Train_x_LB:{}'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    train = pickle.load(filehandle)
with open('../Datapreprocessing/Data/No Smoothing/Test_x_LB:{}'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    test = pickle.load(filehandle)
with open('../Datapreprocessing/Data/No Smoothing/Val_x_LB:{}'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    val = pickle.load(filehandle)
with open('../Datapreprocessing/Data/No Smoothing/Train_y_LB:{}'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    dctr_y = pickle.load(filehandle)
with open('../Datapreprocessing/Data/No Smoothing/Test_y_LB:{}'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    dcte_y = pickle.load(filehandle)
with open('../Datapreprocessing/Data/No Smoothing/Val_y_LB:{}'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    dcva_y = pickle.load(filehandle)

In [8]:
# normalize the data_set
scaler = MinMaxScaler(feature_range=(0, 1))

train = [scaler.fit_transform(np.array(l).reshape(-1,1)) for l in train if l != []]
test = [scaler.fit_transform(np.array(l).reshape(-1,1)) for l in test if l != []]
val = [scaler.fit_transform(np.array(l).reshape(-1,1)) for l in val if l != []]

In [44]:
# convert an array of values into a data_set matrix
def create_data_set(_data_set, _look_back=1, look_forward=10):
    data_x, data_y = [], []
    for i in range(len(_data_set) - _look_back - look_forward):
        a = _data_set[i:(i + _look_back)]
        data_x.append(a)
        data_y.append([_data_set[i + j + _look_back] for j in range(look_forward)])
    return np.array(data_x), np.array(data_y) 

def remove_speaker(data, labels, speaker=0):
    tmp_x = []
    tmp_y = []
    # reshape into X=t and Y=t+n
    for i in range(len(data)):
        if labels[i][speaker] != 1: 
            sentence = data[i]
            x, y = create_data_set(sentence, look_back, look_forward)
            tmp_x.append(x)
            tmp_y.append(y)
    tmp_x = np.array([n for m in tmp_x for n in m], dtype='float16')
    tmp_y = np.array([n for m in tmp_y for n in m], dtype='float16')
    s = tmp_x.shape
#     return tmp_x.reshape(s[0], look_back), tmp_y.reshape(s[0], look_forward)
    return tmp_x, tmp_y.reshape(s[0], look_forward)

In [23]:
train_x, train_y = remove_speaker(train, dctr_y)

In [24]:
val_x, val_y = remove_speaker(val, dcva_y)

In [25]:
test_x, test_y = remove_speaker(test, dcte_y)

In [26]:
train_y.shape

(653877, 10)

In [27]:
# create and fit the LSTM network
patience = 5
callback = [EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto')]

model = Sequential()
model.add(LSTM(units, input_shape=(look_back, 1)))
# model.add(LSTM(units, input_shape=(look_back, 1)))
model.add(Dense(look_forward))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_x, train_y, epochs=50, batch_size=256, verbose=1,  callbacks = callback, validation_data=(val_x, val_y))
model.save('lb{}_lf{}_u{}_full_p{}_ts.h5'.format(look_back, look_forward, units, patience))

Train on 653877 samples, validate on 85285 samples
Epoch 1/50
653877/653877 [==============================] - 3095s 5ms/step - loss: 0.0748 - val_loss: 0.0732
Epoch 2/50
653877/653877 [==============================] - 3132s 5ms/step - loss: 0.0711 - val_loss: 0.0727
Epoch 3/50
653877/653877 [==============================] - 3837s 6ms/step - loss: 0.0702 - val_loss: 0.0718
Epoch 4/50
653877/653877 [==============================] - 2877s 4ms/step - loss: 0.0693 - val_loss: 0.0714
Epoch 5/50
653877/653877 [==============================] - 2899s 4ms/step - loss: 0.0684 - val_loss: 0.0704
Epoch 6/50
653877/653877 [==============================] - 2900s 4ms/step - loss: 0.0676 - val_loss: 0.0701
Epoch 7/50
653877/653877 [==============================] - 2889s 4ms/step - loss: 0.0669 - val_loss: 0.0694
Epoch 8/50
653877/653877 [==============================] - 2879s 4ms/step - loss: 0.0661 - val_loss: 0.0697
Epoch 9/50
653877/653877 [==============================] - 2879s 4ms/step - 

In [45]:
look_back = 100
look_forward = 10
units = 100

In [46]:
with open('../Datapreprocessing/Data/Smoothing/Train_x_LB:{}_s'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    train = pickle.load(filehandle)
with open('../Datapreprocessing/Data/Smoothing/Test_x_LB:{}_s'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    test = pickle.load(filehandle)
with open('../Datapreprocessing/Data/Smoothing/Val_x_LB:{}_s'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    val = pickle.load(filehandle)
with open('../Datapreprocessing/Data/Smoothing/Train_y_LB:{}_s'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    dctr_y = pickle.load(filehandle)
with open('../Datapreprocessing/Data/Smoothing/Test_y_LB:{}_s'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    dcte_y = pickle.load(filehandle)
with open('../Datapreprocessing/Data/Smoothing/Val_y_LB:{}_s'.format(look_back), 'rb') as filehandle:
    # store the data as binary data stream
    dcva_y = pickle.load(filehandle)

In [47]:
# normalize the data_set
scaler = MinMaxScaler(feature_range=(0, 1))

train = [scaler.fit_transform(np.array(l).reshape(-1,1)) for l in train if l != []]
test = [scaler.fit_transform(np.array(l).reshape(-1,1)) for l in test if l != []]
val = [scaler.fit_transform(np.array(l).reshape(-1,1)) for l in val if l != []]

In [48]:
train_x, train_y = remove_speaker(train, dctr_y)

In [49]:
val_x, val_y = remove_speaker(val, dcva_y)

In [50]:
test_x, test_y = remove_speaker(test, dcte_y)

In [51]:
train_x.shape

(284934, 100, 1)

In [52]:
# create and fit the LSTM network
patience = 5
callback = [EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto')]

model = Sequential()
model.add(LSTM(units, input_shape=(look_back, 1)))
# model.add(LSTM(units, input_shape=(look_back, 1)))
model.add(Dense(look_forward))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_x, train_y, epochs=50, batch_size=256, verbose=1,  callbacks = callback, validation_data=(val_x, val_y))
model.save('lb{}_lf{}_u{}_full_p{}_bs_s.h5'.format(look_back, look_forward, units, patience))

Train on 284934 samples, validate on 52502 samples
Epoch 1/50
284934/284934 [==============================] - 313s 1ms/step - loss: 0.0198 - val_loss: 0.0259
Epoch 2/50
284934/284934 [==============================] - 321s 1ms/step - loss: 0.0172 - val_loss: 0.0257
Epoch 3/50
284934/284934 [==============================] - 310s 1ms/step - loss: 0.0171 - val_loss: 0.0175
Epoch 4/50
284934/284934 [==============================] - 301s 1ms/step - loss: 0.0170 - val_loss: 0.0196
Epoch 5/50
284934/284934 [==============================] - 298s 1ms/step - loss: 0.0169 - val_loss: 0.0167
Epoch 6/50
284934/284934 [==============================] - 293s 1ms/step - loss: 0.0168 - val_loss: 0.0199
Epoch 7/50
284934/284934 [==============================] - 295s 1ms/step - loss: 0.0167 - val_loss: 0.0183
Epoch 8/50
284934/284934 [==============================] - 314s 1ms/step - loss: 0.0166 - val_loss: 0.0182
Epoch 9/50
284934/284934 [==============================] - 283s 993us/step - loss: 0